### Capstone Project: The Battle of Neighborhoods (week 1)

For this week, you are required to submit the following: _Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

This submission will eventually become your Introduction/Business Problem section in your final report. So I recommend that you push the report (having your Introduction/Business Problem section only for now) to your Github repository and submit a link to it.

A description of the data and how it will be used to solve the problem. (15 marks)

### 2. Data

In this section a description of the data and how it will be used to solve the problem is provided.

### 2.1 Description of Data

In order to find the most promising borough for John the following data is needed:

1. Average price per m² of the apartments in Munich: This information is gathered through web scrabing from this webpage 'https://de.statista.com/statistik/daten/studie/260438/umfrage/mietpreise-in-muenchen-nach-bezirken/'. Some modifications need to be performed to transform the dataframe in a format that can be used for further analysis. Detailed steps can be found in the Data preparation section.

2.Information about the venues in all boroughs of Munich (including those around Johns home): This information is gathered through web scrabing from this webpage 'https://www.muenchen.de/leben/service/postleitzahlen.html'. As the format does not fit our further data processing requirements, it needs to be transformed. Once the data is in the right format, the Geocoder Python package (https://geocoder.readthedocs.io/index.html) is used to receive the latitude and logitude coordinate for all of the boroughs. The boroughs and their corresponding latitude and longitude are used as input for FourSquare to source information about the boroughs.

### 2.2 How will the data be used to solve the problem

We will start with an exploratory data analysis, where we intend to understand the underlying data. The describe method provides valuable insights for the "average price per m² of the apartments in Munich" investigations.

To get a first impression about the distribution of venues in Munich, they are visualized using Folium map. The chosen color code will give immediate yet superficial insight, how John's favourite venues are distibuted across Munich and how the surrounding of his actual location looks like.

For the further analysis, the venues will be divided into two types: Firstly, we have a dataframe containing all venues of John's personal interest (i.e. parks, coffee stores, restaurants, bars and grocery stores) and one containing all other venues. This subdivision will let us analyse which boroughs are most similar to John's current neighborhood in terms of his personal preferences but also tells us which borough is most similar in terms of other venues. This ensures to find the top borough in terms of John's interests - which is of course the most important criterion - but also in terms of other venues.

One hot encoding and k-means will narrow the list of the most promising boroughs to three. Combining these three with the pricing analysis lets us recommend the best borough match for John.m

### 2.3 Importing Python Libraries

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import itertools

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import geocoder

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Libraries imported.')

Libraries imported.


### 2.4 Data Preprocessing

### a) Get boroughs of munich and corresponding postal code in Dataframe:

First of all,the boroughs of munich and the corresponding postal codes are scraped from the webpage: https://www.muenchen.de/leben/service/postleitzahlen.html and transformed the data into a pandas dataframe as shown below:

In [3]:
url = 'https://www.muenchen.de/leben/service/postleitzahlen.html'
df_munichPoctcode = pd.read_html(url, header=0)[0]
df_munichPoctcode.head()

,Stadtteil,Postleitzahl
0,Allach-Untermenzing,"80995, 80997, 80999, 81247, 81249"
1,Altstadt-Lehel,"80331, 80333, 80335, 80336, 80469, 80538, 80539"
2,Au-Haidhausen,"81541, 81543, 81667, 81669, 81671, 81675, 81677"
3,Aubing-Lochhausen-Langwied,"81243, 81245, 81249"
4,Berg am Laim,"81671, 81673, 81735, 81825"



In order to work better with the data, the following steps are performed on the dataframe:

1.Rename columns: The column names will be translated to english

2.Restructure Data, that we have one single row for every postal code

In [4]:
#Step 1: Rename columns
df_munichPoctcode.rename(columns={'Stadtteil':'Borough', 'Postleitzahl':'PostalCode'}, inplace = True)
df_munichPoctcode.head()

,Borough,PostalCode
0,Allach-Untermenzing,"80995, 80997, 80999, 81247, 81249"
1,Altstadt-Lehel,"80331, 80333, 80335, 80336, 80469, 80538, 80539"
2,Au-Haidhausen,"81541, 81543, 81667, 81669, 81671, 81675, 81677"
3,Aubing-Lochhausen-Langwied,"81243, 81245, 81249"
4,Berg am Laim,"81671, 81673, 81735, 81825"


If you look at the column PostalCode, you'll notice, that there are many postal codes in the PostalCode column separated by a comma, each refering to a different area of the borough. As we would like to compare all of them, we need to get all of these postal codes in a separate row.

In [5]:
#Step 2: Restructure Data

##Save all boroughs in a list
borough_list = df_munichPoctcode['Borough'].to_list()
borough_list = list(itertools.chain.from_iterable(itertools.repeat(x, 9) for x in borough_list))

#Save all Postalcodes in a list
df_postal = df_munichPoctcode[['PostalCode']].PostalCode.str.split(",",expand=True)
postalcode_list = []

count = 0
#Copy all postal codes in one column under each other
for element in (range(0,len(df_postal))):
    postalcode_list.extend(df_postal.iloc[count])
    count+=1
postalcode_list

#Combine borough and postal code list in new dataframe 
df_mucPostcode = pd.DataFrame(list(zip(borough_list, postalcode_list)), columns =['Borough', 'PostalCode']) 
df_mucPostcode.head(20)

#Drop columns that contain nans
df_mucPostcode.dropna(inplace = True)
df_mucPostcode.reset_index(drop=True, inplace=True)
df_mucPostcode.head()

,Borough,PostalCode
0,Allach-Untermenzing,80995
1,Allach-Untermenzing,80997
2,Allach-Untermenzing,80999
3,Allach-Untermenzing,81247
4,Allach-Untermenzing,81249


### b) Get average price per square meter of appartments in the boroughs of munich in Dataframe:

In order to get the price per square meter of appartments in the boroughs of munich the webpage: https://www.tz.de/leben/wohnen/uebersicht-muenchner-mieten-preise-nach-postleitzahlen-tz-6133643.html is scraped and the data is stored in a pandas dataframe as shown below:

In [6]:
url = 'https://www.tz.de/leben/wohnen/uebersicht-muenchner-mieten-preise-nach-postleitzahlen-tz-6133643.html'
df_mucPrice = pd.read_html(url, header=0)[0]
df_mucPrice.head(30)
df_mucPrice.dropna(inplace = True)

df_mucPrice.head()

,PLZ,Miete,Trend,Kaufpreis,Trend.1
1,80995,1410.0,"1,1%",5000,"6,8%"
2,80997,1325.0,"-1,9%",5430,"12,4%"
3,80999,1305.0,"5,2%",5880,"11,4%"
4,81247,1455.0,"2,5%",6520,"1,4%"
5,81249,1325.0,"5,6%",5100,"3,4%"


In order to work better with the data, the following steps are performed on the dataframe:

1.Drop columns that are not needed for further analysis

2.Rename columns: The column names will be translated to english

3.Restructure Data: we would like to have the price per square mether in €instead of Cents

In [7]:
#Step 1: Drop columns
df_mucPrice.drop(["Trend", "Trend.1", "Kaufpreis"],  axis = 1, inplace = True)

In [9]:
#Step 2: Rename columns
df_mucPrice.rename(columns={'PLZ':'PostalCode', 'Miete':'PricePerm2'}, inplace = True)

In [10]:
#Step 3: Restructure Data
df_mucPrice["PricePerm2"] = df_mucPrice["PricePerm2"] /100
df_mucPrice.head()

,PostalCode,PricePerm2
1,80995,14.10
2,80997,13.25
3,80999,13.05
4,81247,14.55
5,81249,13.25


### c) Concat the dataframe a and b to get a single dataframe

In [11]:
df_mucPostcode["PostalCode"] = df_mucPostcode["PostalCode"].astype(int) 
df_mucPrice["PostalCode"] = df_mucPrice["PostalCode"].astype(int) 
df_mucPrice.set_index(['PostalCode'], inplace = True)
df_mucPostcode.set_index(['PostalCode'],inplace = True)

In [12]:
print("There are {} different postal codes in munich. However, we only have prices for {} postal codes".format(df_mucPostcode.shape[0], df_mucPrice.shape[0]))

There are 127 different postal codes in munich. However, we only have prices for 69 postal codes


In [13]:
#Join Dataframes
df_munich = df_mucPostcode.join(df_mucPrice)
df_munich.reset_index(inplace = True)


#Group values based on their borough and calculate mean price
df_munich_grouped = df_munich.groupby('Borough').mean()
df_munich_grouped

##If there is a NaN, replace the NaN with the mean price for this region, calculated in the previous step
pricelist = []

for index in range(0,len(df_munich["PricePerm2"])):
    
    if (np.isnan( df_munich.loc[index,"PricePerm2"] )== True):
        pricelist.append(df_munich_grouped.loc[df_munich.loc[index,"Borough"],"PricePerm2"])
    else:
        pricelist.append(df_munich.loc[index,"PricePerm2"])
    
df_price = pd.DataFrame(pricelist, columns=['New Price'])   
df_muc = df_munich.join(df_price)
df_muc.head()

,PostalCode,Borough,PricePerm2,New Price
0,80331,Altstadt-Lehel,22.3,22.3
1,80333,Altstadt-Lehel,19.1,19.1
2,80333,Altstadt-Lehel,19.1,19.1
3,80333,Maxvorstadt,19.1,19.1
4,80333,Maxvorstadt,19.1,19.1


In [14]:
#Drop column PricePerm2 and make column "new Price" as new priceperm2
df_muc.drop(["PricePerm2"],  axis = 1, inplace = True)
df_muc.rename(columns={'New Price':'PricePerm2'}, inplace = True)
df_muc.head()

,PostalCode,Borough,PricePerm2
0,80331,Altstadt-Lehel,22.3
1,80333,Altstadt-Lehel,19.1
2,80333,Altstadt-Lehel,19.1
3,80333,Maxvorstadt,19.1
4,80333,Maxvorstadt,19.1


In [15]:
#Check if there are NANs in the data
df_muc.isnull().sum(axis = 0)

PostalCode    0
Borough       0
PricePerm2    5
dtype: int64


As it can be seen, there are still 5 rows with NANs in the dataframe. The following rows still contain NANs:

In [16]:
df_muc[df_muc['PricePerm2'].isnull()]

,PostalCode,Borough,PricePerm2
123,81379,Thalkirchen-Obersendling-Fürstenried-Forstenri...,NaN
124,81475,Thalkirchen-Obersendling-Fürstenried-Forstenri...,NaN
125,81476,Thalkirchen-Obersendling-Fürstenried-Forstenri...,NaN
126,81477,Thalkirchen-Obersendling-Fürstenried-Forstenri...,NaN
127,81479,Thalkirchen-Obersendling-Fürstenried-Forstenri...,NaN


As you can see all of the remaining NaN values are in the borough Thalkirchen-Obersendling-Fürstenried-Forstenried-Solln, where we do not have any reference value. That is why these rows are dropped. Thalkirchen-Obersendling-Fürstenried-Forstenried-Solln is not used in the further investigation.

In [17]:
#Drop lines with NANs
df_muc.dropna(subset=['PricePerm2'], inplace = True)

#Check again that there are no NANs in the data
df_muc.isnull().sum(axis = 0)

#reset index
df_muc.reset_index(drop=True, inplace=True)

### d) Get latitude and longitude for each borough

The Geocoder Python package (https://geocoder.readthedocs.io/index.html) is used to get the coordinates (latitude and longitude) for all of the neighborhoods of munich. Therefore a function is defined, that takes as input parameters postalcode and borogh and gives back the latitude and longitude of the specific postalcode and borogh:

In [18]:
##Define function that returns latitude and longitude 
def get_cordinates(postalcode, borogh):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, {}'.format(postalcode, borogh))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude, longitude


#Loop through dataframe, that calls for every row the "get_coordinates" function and adds the latitude and longitude 
##in a new column
for rownr in range(0,df_muc.shape[0]):
    postalcode = df_muc.loc[rownr, 'PostalCode'] 
    borogh = df_muc.loc[rownr, 'Borough'] 
    lat, lon = get_cordinates(postalcode, borogh)
    df_muc.loc[df_muc.index[rownr], 'Latitude'] = lat
    df_muc.loc[df_muc.index[rownr], 'Longitude'] = lon

In [19]:
#check if the latitude and longitude is added in the dataframe
df_muc.head()


,PostalCode,Borough,PricePerm2,Latitude,Longitude
0,80331,Altstadt-Lehel,22.3,48.13487,11.581988
1,80333,Altstadt-Lehel,19.1,48.13641,11.577540
2,80333,Altstadt-Lehel,19.1,48.13641,11.577540
3,80333,Maxvorstadt,19.1,48.14582,11.565170
4,80333,Maxvorstadt,19.1,48.14582,11.565170


Now this dataframe ( _dfmuc ) can be used for data exploration. It will be used to compare the price per square meter of the apartments of the different boroughs.

### e) Get the latitude and longitude of Johns current home

In [20]:
address = 'Gollierstr 6, 80339'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
lat_home = location.latitude
lon_home = location.longitude
print('The geograpical coordinate of Johns current home, Gollierstraße 6 are {}, {}.'.format(lat_home, lon_home))

The geograpical coordinate of Johns current home, Gollierstraße 6 are 48.1358166, 11.5466346.


### f) Use FourSquare to explore the area around the boroughs

To be able to use the Define Foursquare API, the Foresquare Credentials and Version needs to be defines:

In [21]:
CLIENT_ID = 'JUXGOHSTUZBHY4TNZETAMLKLVGXGCVIMN2ASLNCG0TXTRQT2' # your Foursquare ID
CLIENT_SECRET = 'MTE0QUIATNCHWK5YZD3KB50GRTD0HWW2V2NALDXGVOKDOCWC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30 
radius = 500 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JUXGOHSTUZBHY4TNZETAMLKLVGXGCVIMN2ASLNCG0TXTRQT2
CLIENT_SECRET:MTE0QUIATNCHWK5YZD3KB50GRTD0HWW2V2NALDXGVOKDOCWC


A function is defined, that takes as input the borough as well as the latitude and longitude and gives back the venues around the location provided:

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
muc_venues = getNearbyVenues(names=df_muc['Borough'],
                                   latitudes=df_muc['Latitude'],
                                   longitudes=df_muc['Longitude']
                                   )

Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Maxvorstadt
Maxvorstadt
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Ludwigsvorstadt-Isarvorstadt
Ludwigsvorstadt-Isarvorstadt
Ludwigsvorstadt-Isarvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Schwanthalerhöhe
Schwanthalerhöhe
Schwanthalerhöhe
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Ludwigsvorstadt-Isarvorstadt
Ludwigsvorstadt-Isarvorstadt
Ludwigsvorstadt-Isarvorstadt
Sendling
Sendling
Sendling
Ludwigsvorstadt-Isarvorstadt
Sendling
Schwanthalerhöhe
Altstadt-Lehel
Altstadt-Lehel
Ludwigsvorstadt-Isarvorstadt
Ludwigsvorstadt-Isarvorstadt
Sendling
Sendling
Altstadt-Lehel
Schwabing-Freimann
Altstadt-Lehel
Altstadt-Lehel
Maxvorstadt
Maxvorstadt
Neuhausen-Nymphenburg
Maxvorstadt
Maxvorstadt
Neuhausen-Nymphenburg
Neuhausen-Nymphenburg
Moosach
Moosach
Neuhausen-Nymphenburg
Neuhausen-Nymphenburg
Moosach
Moosach
Neuhausen-Nymphenburg
Neuhausen-Nymphenburg
Neuhausen-Nymphenburg
Laim
Sendling-Westpark
Laim
Pasing-Obermenzing
Hadern
Hadern
Laim
Laim
P

In [27]:
muc_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altstadt-Lehel,48.13487,11.581988,Little London,48.135562,11.580961,Steakhouse
1,Altstadt-Lehel,48.13487,11.581988,Gasthaus Isarthor,48.134928,11.583601,German Restaurant
2,Altstadt-Lehel,48.13487,11.581988,OOH BABY I LIKE IT RAW,48.134023,11.580400,Café
3,Altstadt-Lehel,48.13487,11.581988,Mandarin Oriental Munich,48.137257,11.580779,Hotel
4,Altstadt-Lehel,48.13487,11.581988,Bar Centrale,48.136821,11.579873,Café


For the further analysis, we now have the following data(frames):

1.df_muc: Contains the average price per m² of the apartments in Munich. This dataframe is used to compare the average price per square meter of the boroughs in Munich with the actual price per square meter of Johns apartment.

2.muc_venues: Contains all the venues in the boroughs of munich. This dataframe is used to find borrouoghs that fits most Johns requirements.
In addition to that, we know that Johns home is in Gollierstr 6, 80339 with latitude and longitude of 48.1358166 and 11.5466346.